이번 챕터에서는 소프트맥스 회귀를 로우-레벨과 F.cross_entropy를 사용해서 구현해보겠습니다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
#훈련 데이터와 레이블을 텐서로 선언합니다.
#array -> Tensor
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]] #|x_train|= (8, 4). #of samples = 8 . #of features = 4
y_train = [2, 2, 2, 1, 1, 1, 0, 0] #|y_train| = (8,) 
#y_train은 각 샘플에 대한 레이블인데, 여기서는 0, 1, 2의 값을 가지는 것으로 보아 총 3개의 클래스가 존재
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

### 1. 소프트맥스 회귀 구현(로우-레벨)

In [4]:
#x_train의 크기와 y_train의 크기를 확인
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [5]:
'''
x_train의 크기는 8 × 4이며, y_train의 크기는 8 × 1입니다. 
그런데 최종 사용할 레이블은 y_train에서 원-핫 인코딩을 한 결과이어야 합니다.(low-level이라 F.cross_entropy안쓸경우)
클래스의 개수는 3개이므로 y_train에 원-핫 인코딩한 결과는 8 × 3의 개수를 가져야 합니다.
'''
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)


torch.Size([8, 3])


In [9]:
'''
y_train에서 원-핫 인코딩을 한 결과인 y_one_hot의 크기는 8 × 3입니다.
XW + B -> Y_one_hot |X|=(8,4) |B|=|Y_one_hot|=(8,3)이므로
즉, W 행렬의 크기는 4 × 3이어야 합니다.
W와 b를 선언하고, 옵티마이저로는 경사 하강법을 사용합니다. 
그리고 학습률은 0.1로 설정합니다.(Tip: lr의 경우, 사용하는 모델의 논문에서 사용한 lr찾아보기)
'''
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True) #BroadCasting되어 (8,3)된당... 맞낭...??
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)


In [13]:
#F.softmax()와 torch.log()를 사용하여 가설과 비용 함수를 정의하고,
#총 1,000번의 에포크를 수행합니다.

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  #가설
  hypothesis = F.softmax(x_train.matmul(W)+ b, dim = 1)

  #Cost function
  cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

  #cost로 hypothesis 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  #100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.314640
Epoch  200/1000 Cost: 0.178421
Epoch  300/1000 Cost: 0.122975
Epoch  400/1000 Cost: 0.093058
Epoch  500/1000 Cost: 0.074548
Epoch  600/1000 Cost: 0.062041
Epoch  700/1000 Cost: 0.053055
Epoch  800/1000 Cost: 0.046302
Epoch  900/1000 Cost: 0.041049
Epoch 1000/1000 Cost: 0.036850


## 2. 소프트맥스 회귀 구현하기(하이-레벨)

이제는 F.cross_entropy()를 사용하여 비용 함수를 구현해보겠습니다. **주의할 점은 F.cross_entropy()는 그 자체로 소프트맥스 함수를 포함하고 있으므로 가설에서는 소프트맥스 함수를 사용할 필요가 없습니다.**

위와 동일한 x_train과 y_train을 사용합니다.

In [15]:
#훈련 데이터와 레이블을 텐서로 선언합니다.
#array -> Tensor
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]] #|x_train|= (8, 4). #of samples = 8 . #of features = 4
y_train = [2, 2, 2, 1, 1, 1, 0, 0] #|y_train| = (8,) 
#y_train은 각 샘플에 대한 레이블인데, 여기서는 0, 1, 2의 값을 가지는 것으로 보아 총 3개의 클래스가 존재
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [16]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  #Cost 계산
  z = x_train.matmul(W) + b #hypotheis
  #주의할 점은 F.cross_entropy()는 그 자체로 소프트맥스 함수를 포함하고 있으므로 
  #가설에서는 소프트맥스 함수를 사용할 필요가 없습니다.
  cost = F.cross_entropy(z, y_train) #원-핫 벡터를 넣을 필요없이 바로 실제값을 인자로 사용
  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## 3. 소프트맥스 회귀 nn.Module로 구현하기

이번에는 nn.Module로 소프트맥스 회귀를 구현해봅시다. 선형 회귀에서 구현에 사용했던 **nn.Linear()를 사용**합니다. output_dim이 1이었던 선형 회귀때와 달리 output_dim은 이제 클래스의 개수여야 합니다.

In [17]:
# 모델을 선언 및 초기화. 4개의 features(특성)을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3.
model = nn.Linear(4, 3)

#아래에서 F.cross_entropy()를 사용할 것이므로 따로 소프트맥스 함수를 가설에 정의하지 않습니다.
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))


Epoch    0/1000 Cost: 1.655176
Epoch  100/1000 Cost: 0.707168
Epoch  200/1000 Cost: 0.622998
Epoch  300/1000 Cost: 0.564230
Epoch  400/1000 Cost: 0.512928
Epoch  500/1000 Cost: 0.464686
Epoch  600/1000 Cost: 0.417831
Epoch  700/1000 Cost: 0.371580
Epoch  800/1000 Cost: 0.325683
Epoch  900/1000 Cost: 0.281097
Epoch 1000/1000 Cost: 0.246529


## 4. 소프트맥스 회귀 클래스로 구현하기

이제 소프트맥스 회귀를 nn.Module을 상속받은 클래스로 구현해봅시다.

In [20]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Outputdim이 3!

    def forward(self, x):
        return self.linear(x)

model = SoftmaxClassifierModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.772288
Epoch  100/1000 Cost: 0.689931
Epoch  200/1000 Cost: 0.607083
Epoch  300/1000 Cost: 0.549071
Epoch  400/1000 Cost: 0.498127
Epoch  500/1000 Cost: 0.450025
Epoch  600/1000 Cost: 0.403168
Epoch  700/1000 Cost: 0.356838
Epoch  800/1000 Cost: 0.310965
Epoch  900/1000 Cost: 0.267557
Epoch 1000/1000 Cost: 0.240278
